<a href="https://colab.research.google.com/github/hissain/mlworks/blob/main/codes/Bidirectional_RNN_from_scratch_M2O.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import
import torch
import torch.nn as nn
import math

In [ ]:
# Seed
torch.manual_seed(0)

In [ ]:
class RNNLayer(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNNLayer, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Weight matrices for input and hidden layer connections
        self.W_xh = torch.nn.Parameter(torch.randn(input_size, hidden_size))
        self.W_hh = torch.nn.Parameter(torch.randn(hidden_size, hidden_size))
        # Bias term for hidden layer
        self.b_h = torch.nn.Parameter(torch.zeros(hidden_size))

    def forward(self, input_data, hidden_state=None):
        """
        Performs a forward pass through the RNN layer.

        Args:
            input_data: A tensor of shape (batch_size, input_size) representing the input sequence.
            hidden_state: A tensor of shape (batch_size, hidden_size) representing the initial hidden state (optional).

        Returns:
            output: A tensor of shape (batch_size, hidden_size) representing the hidden state at each time step.
            hidden_state: A tensor of shape (batch_size, hidden_size) representing the hidden state.
        """
        batch_size, _ = input_data.size()

        # Initialize hidden state if not provided
        if hidden_state is None:
            hidden_state = torch.zeros(batch_size, self.hidden_size)

        # Calculate current hidden state
        hidden_state = torch.tanh(
            # (batch_size, input_size) x (input_size, hidden_size)
            # = (batch_size, hidden_size)
            torch.mm(input_data, self.W_xh) + \
            # (batch_size, hidden_size) x (hidden_size, hidden_size)
            # = (batch_size, hidden_size)
            torch.mm(hidden_state, self.W_hh) + \
            # hidden_size
            self.b_h
        )

        return hidden_state

In [ ]:
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.forward_rnn_cell = RNNLayer(input_size, hidden_size)
        self.backward_rnn_cell = RNNLayer(input_size, hidden_size)
        self.fc = torch.nn.Linear(2 * hidden_size, output_size)

    def forward(self, inputs):
        """
        Performs a forward pass through the RNN model.

        Args:
            inputs: A tensor of shape (batch_size, seq_len, input_size) representing the input sequence.

        Returns:
            prediction: A tensor of shape (batch_size, output_size) representing the model output.
        """
        forward_hidden_state = None
        backward_hidden_state = None

        _, seq_len, _ = inputs.size()

        for t in range(seq_len):
            forward_hidden_state = self.forward_rnn_cell(inputs[:, t, :], forward_hidden_state)
            backward_hidden_state = self.backward_rnn_cell(inputs[:, seq_len - t - 1, :], backward_hidden_state)

        hidden_state = torch.cat([forward_hidden_state, backward_hidden_state], dim=1)
        prediction = self.fc(hidden_state)

        return prediction, hidden_state

In [ ]:
# Example usage:
input_size = 10
output_size = 10
hidden_size = 20
seq_length = 5
batch_size = 2

# Create LSTM model
rnn = BiRNN(input_size, hidden_size, output_size)

# Generate some random input data
input_data = torch.randn(batch_size, seq_length, input_size)

# Forward pass
output, hidden_state_last = rnn(input_data)
print("Output shape:", output.shape)
print("Last hidden state shape:", hidden_state_last.shape)


Output shape: torch.Size([2, 10])
Last hidden state shape: torch.Size([2, 40])
